This file defines functions for connecting with a Gravwell system.

You'll probably want to redefine GravwellToken and GravwellServer -- you can do this in your own notebooks, or you can modify this one. The defaults point at a demo Gravwell instance with some basic data.

GravwellServer should be the base URL of your Gravwell instance.

To "import" this code, add a code block at the top of your notebook containing `%run gravwell-lib.ipynb`.

In [13]:
# CHANGE ME
GravwellToken = "clGSxt3bOwtVCqqNiIF6I5R8RoU_6nK51X3R7Eg3a-edMdY-X7A"
GravwellServer = "https://demo.gravwell.io"

from io import StringIO
import matplotlib.pyplot as plt
import pandas as pd
import requests
import urllib
import json
import datetime
import pytz

def query(query, duration='-1h', start='', end=''):
    """
    Run a Gravwell query and return the results in a pandas.DataFrame
    
    Parameters
    -----------
    query: str
        The Gravwell query to run.
        
    duration: str, default '-1h'
        The duration over which the query should run, e.g. "-24h". 
        If `start` and `end` are specified, duration will be ignored.
    
    start, end: str, default ''
        Specifies explicit start and end points for the query.
        Should be an RFC3339/ISO8601 timestamp e.g. "2006-01-02T15:04:05Z" or "2006-01-02T15:04:05-07:00".
        Overrides `duration`.
        This example runs a search from July 1, 2022 00:00:00 (local time) to the current time:
        
            end=datetime.datetime.now().astimezone().isoformat()
            start=datetime.datetime.fromisoformat('2022-07-01').astimezone().isoformat()
            query("tag=stocks stats count", start=start, end=end)

    """
    headers = {
        "Gravwell-Token": GravwellToken,
        "query": query,
        "duration": duration,
        "start": start,
        "end": end,
        "format": "csv"
    }
    if start != '' and end != '':
        headers["duration"] = ''
    r = requests.post(urllib.parse.urljoin(GravwellServer, '/api/search/direct'), headers=headers)
    r.raise_for_status()
    df = pd.DataFrame()
    if len(r.text) > 0:
        df = pd.read_csv(StringIO(r.text), parse_dates=[0], infer_datetime_format=True)
    return df